# Day 10

In [23]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

enum class Direction(val shift: Position) {
    Up(Position(0, -1)),
    Left(Position(-1, 0)),
    Right(Position(1, 0)),
    Down(Position(0, 1))
}

enum class TileType(val symbol: Char, val directions: Set<Direction>) {
    VerticalPipe('|', setOf(Direction.Up, Direction.Down)),
    HorizontalPipe('-', setOf(Direction.Left, Direction.Right)),
    NorthEastBendPipe('L', setOf(Direction.Up, Direction.Right)),
    NorthWestBendPipe('J', setOf(Direction.Up, Direction.Left)),
    SouthWestBendPipe('7', setOf(Direction.Left, Direction.Down)),
    SouthEastBestPipe('F', setOf(Direction.Right, Direction.Down)),
    StartingPosition('S', Direction.entries.toSet()),
    Ground('.', emptySet()),
}

data class Position(
    val x: Int,
    val y: Int
) {
    operator fun plus(other: Position): Position =
        Position(x + other.x, y + other.y)
}

data class Tile(
    val type: TileType,
    val position: Position,
    val marked: Boolean = false
)

val tilesFromSymbols = TileType.entries.associate { it.symbol to it }
fun Char.toTile(x: Int, y: Int): Tile =
    tilesFromSymbols[this]
        ?.let { type -> Tile(type = type, position = Position(x, y)) }
        ?: throw RuntimeException("No tile type associated with the symbol $this")

val lines = Path("./input").readLines().filter { it.isNotBlank() }
val tiles = lines.flatMapIndexed { y, line ->
    line.mapIndexed { x, char ->
        char.toTile(x, y)
    }
}


typealias TileMap = Map<Position, Tile>

val map: TileMap = tiles.associateBy { it.position }
val start = tiles.first { it.type == TileType.StartingPosition }

fun Tile.connectedPipes(map: TileMap): Set<Tile> =
    type.directions
        .mapNotNull { map[position + it.shift] }
        .toSet()

fun Tile.neighbours(map: TileMap): Set<Tile> =
    Direction.entries
        .mapNotNull { map[position + it.shift] }
        .toSet()

tailrec fun findLoop(start: Tile, map: TileMap, visitedNodes: List<Tile> = emptyList()): List<Tile> {
    val connections = start.connectedPipes(map).filter { it.connectedPipes(map).contains(start) }
    val availableConnections = connections - visitedNodes

    // Reached the full loop
    if (availableConnections.size == 0 && connections.any { it.type == TileType.StartingPosition }) {
        return visitedNodes + start
    }

    return findLoop(availableConnections.random(), map, visitedNodes + start)
}

// Part 1 
val loop = findLoop(start, map)

println("Part 1" to loop.size / 2)

// Part 2

fun Tile.zoom(): List<Tile> {
    val pattern = when (type) {
        TileType.VerticalPipe -> listOf(
            listOf('.', '|', '.'),
            listOf('.', '|', '.'),
            listOf('.', '|', '.'),
        )

        TileType.HorizontalPipe -> listOf(
            listOf('.', '.', '.'),
            listOf('-', '-', '-'),
            listOf('.', '.', '.'),
        )

        TileType.NorthEastBendPipe -> listOf(
            listOf('.', '|', '.'),
            listOf('.', '|', '.'),
            listOf('.', '|', '.'),
        )

        TileType.NorthWestBendPipe -> listOf(
            listOf('.', '|', '.'),
            listOf('-', 'J', '.'),
            listOf('.', '.', '.'),
        )

        TileType.SouthWestBendPipe -> listOf(
            listOf('.', '.', '.'),
            listOf('-', '7', '.'),
            listOf('.', '|', '.'),
        )

        TileType.SouthEastBestPipe -> listOf(
            listOf('.', '.', '.'),
            listOf('.', 'F', '-'),
            listOf('.', '|', '.'),
        )

        TileType.StartingPosition -> listOf(
            listOf('.', '|', '.'),
            listOf('-', 'S', '-'),
            listOf('.', '|', '.'),
        )

        TileType.Ground -> listOf(
            listOf('.', '.', '.'),
            listOf('.', '.', '.'),
            listOf('.', '.', '.'),
        )
    }

    return pattern.flatMapIndexed { x, line ->
        line.mapIndexed { y, char ->
            char.toTile(x + position.x, y + position.y)
        }
    }
}

fun TileMap.zoom(): TileMap =
    values.flatMap { it.zoom() }
        .associateBy { it.position }

tailrec fun TileMap.fill(origin: Tile): TileMap {
    println("Filling: $origin")
    
    val updated = this + (origin.position to origin.copy(marked = true))
    val fillableNeighbours = origin.neighbours(this)
        .filter { it.type == TileType.Ground }
        .filterNot { it.marked }
    
    
    if (fillableNeighbours.isEmpty()) {
        return updated
    }
    
    
    return updated.fill(fillableNeighbours.random())
}

val zoomedMap = map.zoom()
val origin = map.values.first { it.type == TileType.Ground }

zoomedMap.fill(origin).values.count { it.marked }

(Part 1, 6701)
Filling: Tile(type=Ground, position=Position(x=11, y=0), marked=false)
Filling: Tile(type=Ground, position=Position(x=10, y=0), marked=false)
Filling: Tile(type=Ground, position=Position(x=9, y=0), marked=false)
Filling: Tile(type=Ground, position=Position(x=9, y=1), marked=false)
Filling: Tile(type=Ground, position=Position(x=10, y=1), marked=false)
Filling: Tile(type=Ground, position=Position(x=10, y=2), marked=false)
Filling: Tile(type=Ground, position=Position(x=9, y=2), marked=false)
Filling: Tile(type=Ground, position=Position(x=9, y=3), marked=false)
Filling: Tile(type=Ground, position=Position(x=9, y=4), marked=false)
Filling: Tile(type=Ground, position=Position(x=9, y=5), marked=false)
Filling: Tile(type=Ground, position=Position(x=8, y=5), marked=false)
Filling: Tile(type=Ground, position=Position(x=8, y=4), marked=false)
Filling: Tile(type=Ground, position=Position(x=8, y=3), marked=false)
Filling: Tile(type=Ground, position=Position(x=8, y=2), marked=false)
F

70